In [1]:
import os

In [2]:
import cv2

In [3]:
from keras.preprocessing import image

In [4]:
categories = ['with_mask', 'without_mask']

In [5]:
data = []
for category in categories:
    path = os.path.join('train', category)
    
    label = categories.index(category)
    
    for file in os.listdir(path):
        
        img_path = os.path.join(path,file)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))
        
        data.append([img,label])

In [6]:
len(data)

1508

In [7]:
import random

In [8]:
random.shuffle(data)

In [9]:
x = []
y = []

for features, label in data:
    x.append(features)
    y.append(label)

In [10]:
len(x)

1508

In [11]:
len(y)

1508

In [12]:
import numpy as np

In [13]:
x = np.array(x)
y = np.array(y)

In [14]:
X = x/255

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [17]:
print(X_train.shape)
print(X_test.shape)

(1206, 224, 224, 3)
(302, 224, 224, 3)


In [18]:
from keras.applications.vgg16 import VGG16

In [19]:
vgg = VGG16()

In [20]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [21]:
#removing last line and adding our layer
from keras import Sequential

In [42]:
model = Sequential()

In [43]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [25]:
for layer in model.layers:
    layer.trainable=False

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [27]:
#adding last layer
from keras.layers import Dense

In [28]:
model.add(Dense(1, activation='sigmoid'))

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [30]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
model.fit(X_train,y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
38/38 [==============================] - 345s 9s/step - loss: 0.6508 - accuracy: 0.5813 - val_loss: 0.4866 - val_accuracy: 0.7450
Epoch 2/5
38/38 [==============================] - 331s 9s/step - loss: 0.3852 - accuracy: 0.8740 - val_loss: 0.3250 - val_accuracy: 0.9007
Epoch 3/5
38/38 [==============================] - 331s 9s/step - loss: 0.3008 - accuracy: 0.9030 - val_loss: 0.2479 - val_accuracy: 0.9205
Epoch 4/5
38/38 [==============================] - 329s 9s/step - loss: 0.2420 - accuracy: 0.9328 - val_loss: 0.2139 - val_accuracy: 0.9305
Epoch 5/5
38/38 [==============================] - 328s 9s/step - loss: 0.2137 - accuracy: 0.9370 - val_loss: 0.1835 - val_accuracy: 0.9536


In [32]:
cap = cv2.VideoCapture(0)

In [49]:
def detect_face_mask(img):
    y_pred = model.predict(img.reshape(1,224,224,3))
    return y_pred[0][0]

In [54]:
samp1 = cv2.imread('sample/2.jpg')
samp1 = cv2.resize(samp1,(224,224))

In [55]:
detect_face_mask(samp1)

1/1 [==============================] - 0s 183ms/step


0.0

In [56]:
def draw_label(img,text,pos,bg_color):
    text_size = cv2.getTextSize(text,cv2.FONT_HERSHEY_PLAIN,1,cv2.FILLED)
    
    end_x = pos[0] + text_size[0][0] + 2
    end_y = pos[1] + text_size[0][1] - 2
    
    cv2.rectangle(img,pos,(end_x, end_y), bg_color,cv2.FILLED)
    cv2.putText(img, text,pos,cv2.FONT_HERSHEY_PLAIN, 1,(0,0,0), 1, cv2.LINE_AA)

In [71]:
def detect_face(img):
    cods = haar.detectMultiScale(img)
    return cods

In [74]:
while True:
    
    ret, frame = cap.read()
    
    #call detection function
    img = cv2.resize(frame,(224,224))
    
    y_pred = detect_face_mask(img)
    
    cods = detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))
    
    for x,y,w,h in cods:
        cv2.rectangle(frame,(x,y),(x+w, y+h), (200,100,0),3)
        
    
    draw_label(frame, 'Face mask detection', (20,20),(255,0,0))
    
    if y_pred ==0:
        draw_label(frame, 'Mask', (20,50),(0,255,0))
    else:
        draw_label(frame, 'No Mask', (20,50),(0,0,255))
        
        
    
    cv2.imshow('window', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break
        
cv2.destroyAllWindows()

1/1 [==============================] - 0s 354ms/step


1/1 [==============================] - 0s 332ms/step


In [72]:
while True:
    
    ret, frame = cap.read()
    
    cods = detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))
    
    for x,y,w,h in cods:
        cv2.rectangle(frame,(x,y),(x+w, y+h), (200,100,0),3)
    
    cv2.imshow('window', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break
        
cv2.destroyAllWindows()

In [67]:
haar = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')